In [58]:
import folium
from folium import CircleMarker
import geopandas as gpd
import xyzservices as xyz
import os
import re
import importlib
import models
importlib.reload(models)
from models import get_conn
import pandas as pd
import elkai
pd.set_option('display.expand_frame_repr', False) # display full data in terminal
engine, Session = get_conn()

In [39]:
with Session() as session:
    waypoints = gpd.read_postgis("SELECT * FROM waypoints WHERE notes  = '2023 2024 Van Trip'", con=engine, geom_col='geometry')

# Travelling Salseman

# Move 'Denver Colorado' to the first position in waypoints
denver = waypoints[waypoints.title == 'Denver Colorado']
not_denver = waypoints[waypoints.title != 'Denver Colorado']
places = pd.concat([denver, not_denver]).reset_index(drop=True)

waypoints_dict = {}
for _, row in places.iterrows():
    waypoints_dict[row.title] = (row.latitude, row.longitude)

runs = 1

solved_cities = elkai.Coordinates2D(waypoints_dict).solve_tsp(runs=runs)


In [97]:
# draw cities
center = [44.2, -102]
map = folium.Map(location=center, zoom_start=4.5)

for _, point in waypoints.iterrows():
    folium.Marker(
        location = [point.latitude, point.longitude],
        # radius=4,
        color='blue',
        # weight=1,
        # fill=False,
        # fill_color='blue',
        # fill_opacity=0.2,
        tooltip=point.title,
    ).add_to(map)
map

def escape_placeholders(url, placeholders):
    # Escape curly braces around the specified placeholders
    for placeholder in placeholders:
        url = re.sub(rf'\{{({placeholder})\}}', r'{{\1}}', url)
    return url
# Map Layers

# Stadia Watercolor map
water_color = xyz.providers.Stadia.StamenWatercolor
stadia_api_key = os.getenv('STADIA_API_KEY')
base_url = water_color.url
water_color_url = escape_placeholders(base_url, ['z', 'x', 'y'])
substituted_url = water_color_url.format(variant='stamen_watercolor', ext='jpg') + '?api_key=' + stadia_api_key
stadia_attr = water_color.attribution
folium.TileLayer(
    tiles=substituted_url,
    attr=stadia_attr,
    name='Watercolor',
    overlay=True,
).add_to(map)

# Open Topo layer
folium.TileLayer('Open Topo', name='Open Topo').add_to(map)

# add layer control
folium.LayerControl().add_to(map)

# Draw TSP route
def draw_tsp_route(map):
    ordered_indices = [waypoints[waypoints.title == title].index[0] for title in solved_cities]
    ordered_waypoints = waypoints.loc[ordered_indices].reset_index(drop=True)

    coords = [[w.latitude, w.longitude] for _, w in ordered_waypoints.iterrows()]

    folium.PolyLine(
        locations=coords,
        color='red',
        weight=2,
        opacity=1
    ).add_to(map)

# draw_tsp_route(map)
# map.save('van_trip_map_watercolor.html')
map